<a href="https://colab.research.google.com/github/shayanahmad7/Weather-Mood-TaylorSwift-Playlist/blob/main/apifeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install requests spotipy

In [ ]:
import os
import requests
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import random
from google.colab import userdata

# Accessing API keys from the Colab secrets tab
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')  # Access OpenWeatherMap API Key
SPOTIFY_CLIENT_ID = userdata.get('SPOTIFY_CLIENT_ID')  # Access Spotify Client ID
SPOTIFY_CLIENT_SECRET = userdata.get('SPOTIFY_CLIENT_SECRET')  # Access Spotify Client Secret
SPOTIFY_REDIRECT_URI = 'http://localhost:8000' #Redirect to a blank page After Spotify Login



# OpenWeatherMap API setup
WEATHER_API_URL = "https://api.openweathermap.org/data/2.5/weather"

# Spotify API setup: Using Client Credentials to access Spotify data
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID,
                                                           client_secret=SPOTIFY_CLIENT_SECRET))

def get_weather(city):
    """
    Fetches weather data for a given city using the OpenWeatherMap API.
    The temperature and weather condition are then used to determine the user's mood.
    """
    params = {
        "q": city,  # The city entered by the user
        "appid": WEATHER_API_KEY,  # API key for OpenWeatherMap
        "units": "metric"  # Returning temperature in Celsius
    }

    # Send request to OpenWeatherMap API and handle errors
    print(f"Sending request to: {WEATHER_API_URL} for city: {city}")
    try:
        response = requests.get(WEATHER_API_URL, params=params)
        print(f"Response status code: {response.status_code}")

        if response.status_code == 200:
            return response.json()  # Return weather data if successful
        else:
            print(f"Error fetching weather data: {response.text}")
            raise Exception(f"Error: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        raise

def get_mood_from_weather(weather_data):
    """
    Determines the user's mood based on the current temperature and weather conditions.
    Uses a simple heuristic to map weather to moods.
    """
    temp = weather_data["main"]["temp"]  # Current temperature
    weather_condition = weather_data["weather"][0]["main"].lower()  # Weather description

    print(f"Temperature: {temp}°C, Weather condition: {weather_condition}")

    # Mapping weather conditions to moods
    if "clear" in weather_condition:
        return "happy" if temp > 20 else "calm"
    elif "cloud" in weather_condition:
        return "reflective"
    elif "rain" in weather_condition:
        return "melancholic"
    elif "snow" in weather_condition:
        return "cozy"
    else:
        return "energetic"

def get_taylor_swift_songs(mood, num_songs=15):
    """
    Fetches Taylor Swift's songs from Spotify and filters them based on the given mood.
    Returns a randomly selected list of songs that match the user's mood.
    """
    taylor_swift_id = "06HL4z0CvFAxyc27GXpf02"  # Taylor Swift's Spotify artist ID

    try:
        print(f"Fetching Taylor Swift's albums from Spotify...")
        albums = sp.artist_albums(taylor_swift_id, album_type='album', limit=50)['items']
        print(f"Found {len(albums)} albums")

        all_tracks = []
        for album in albums:

            tracks = sp.album_tracks(album['id'])['items']
            all_tracks.extend(tracks)

        print(f"Total tracks fetched: {len(all_tracks)}")

        # Collecting track IDs for fetching audio features
        track_ids = [track['id'] for track in all_tracks]

        # Fetching audio features for each track in batches of 100
        audio_features = []
        for i in range(0, len(track_ids), 100):
            batch = track_ids[i:i+100]
            audio_features.extend(sp.audio_features(batch))

        print(f"Total audio features fetched: {len(audio_features)}")

        # Create a dictionary of track ID to its audio features
        track_features = {feature['id']: feature for feature in audio_features if feature}

        # Stricter mood-matching conditions based on energy and valence
        mood_features = {
            "happy": {"min_valence": 0.6, "min_energy": 0.6},
            "calm": {"max_energy": 0.3, "max_valence": 0.6},
            "reflective": {"min_valence": 0.2, "max_valence": 0.5, "max_energy": 0.4},
            "melancholic": {"max_valence": 0.3, "max_energy": 0.3},
            "cozy": {"min_valence": 0.2, "max_energy": 0.5},
            "energetic": {"min_energy": 0.7}
        }

        # Filtering tracks that match the mood
        matching_tracks = []
        for track in all_tracks:
            features = track_features.get(track['id'])
            if features:
                if all(features.get(key[4:], 0) >= value for key, value in mood_features[mood].items() if key.startswith("min_")) and \
                   all(features.get(key[4:], 1) <= value for key, value in mood_features[mood].items() if key.startswith("max_")):
                    matching_tracks.append({
                        "title": track.get('name', 'Unknown Title'),
                        "album": track.get('album', {}).get('name', 'Unknown Album'),
                        "spotify_url": f"https://open.spotify.com/track/{track['id']}"
                    })

        print(f"Total tracks matching mood '{mood}': {len(matching_tracks)}")

        # Randomly select 10-15 tracks from the matching tracks
        selected_tracks = random.sample(matching_tracks, min(len(matching_tracks), num_songs))

        return selected_tracks

    except Exception as e:
        print(f"Error fetching Taylor Swift songs: {e}")
        return []

def create_playlist(city):
    """
    Creates a playlist of Taylor Swift songs based on the user's current weather and mood.
    """
    try:
        weather_data = get_weather(city)  # Get weather data
        mood = get_mood_from_weather(weather_data)  # Determine mood
        print(f"Determined mood: {mood}")
        songs = get_taylor_swift_songs(mood, num_songs=15)  # Get matching songs

        # Playlist details
        playlist_name = f"Taylor Swift {mood.capitalize()} Weather Playlist for {city}"

        return {
            "city": city,
            "temperature": weather_data["main"]["temp"],
            "weather": weather_data["weather"][0]["description"],
            "mood": mood,
            "playlist_name": playlist_name,
            "songs": songs
        }
    except Exception as e:
        print(f"Error creating playlist: {e}")
        return {"error": str(e)}

# Main execution: User enters a city, and a playlist is generated based on the weather
city = input("Enter a city name: ")
result = create_playlist(city)

if "error" in result:
    print(f"Error: {result['error']}")
else:
    print(f"Weather in {result['city']}: {result['temperature']}°C, {result['weather']}")
    print(f"Mood: {result['mood']}")
    playlist_name = result['playlist_name']  # Store playlist_name as a global variable
    print(f"\nSuggested playlist: {result['playlist_name']}")
    print("\nSongs in the playlist:")
    for song in result['songs']:
        print(f"- {song['title']} (Album: {song['album']})")
        print(f"  Listen on Spotify: {song['spotify_url']}")
    songs = result['songs']

# Authenticate with Spotify
sp_oauth = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIFY_CLIENT_ID,
                                               client_secret=SPOTIFY_CLIENT_SECRET,
                                               redirect_uri=SPOTIFY_REDIRECT_URI,
                                               scope='playlist-modify-public'))

# Get the authorization URL
auth_url = sp_oauth.auth_manager.get_authorize_url()
print(f"Please visit this URL to authorize the app: {auth_url}")

# After authorizing, paste the code from the redirect URL here
code = input("Enter the authorization code (you can find it in the Address Bar after code=): ")

# Get the access token using the code
sp_oauth.auth_manager.get_access_token(code)

# Get the current user's ID
user_id = sp_oauth.current_user()['id']

# Create a new playlist
playlist = sp_oauth.user_playlist_create(user=user_id, name=playlist_name, public=True)
playlist_id = playlist['id']

# Add songs to the playlist
song_ids = [song['spotify_url'].split('/')[-1] for song in result['songs']] # Extract song IDs from the result dictionary
sp_oauth.playlist_add_items(playlist_id, song_ids) # Pass the list of song IDs to the function

print(f'Playlist "{playlist_name}" created successfully!')

print("\nNote: This script respects API rate limits. If you encounter any errors, please wait before trying again.")

Enter a city name: Los Angeles
Sending request to: https://api.openweathermap.org/data/2.5/weather for city: Los Angeles
Response status code: 200
Temperature: 19.64°C, Weather condition: clear
Determined mood: calm
Fetching Taylor Swift's albums from Spotify...
Found 29 albums
Total tracks fetched: 577
Total audio features fetched: 577
Total tracks matching mood 'calm': 56
Weather in Los Angeles: 19.64°C, clear sky
Mood: calm

Suggested playlist: Taylor Swift Calm Weather Playlist for Los Angeles

Songs in the playlist:
- Vigilante Shit (Album: Unknown Album)
  Listen on Spotify: https://open.spotify.com/track/0GKDhq6ZbmSbRHd3eyGlB7
- hoax (Album: Unknown Album)
  Listen on Spotify: https://open.spotify.com/track/4XCYmu7QLySdIahgBALKJK
- my tears ricochet (Album: Unknown Album)
  Listen on Spotify: https://open.spotify.com/track/0Oju1YcbPCJuUQQWoZNdJK
- peace (Album: Unknown Album)
  Listen on Spotify: https://open.spotify.com/track/0O9i6jhscByEJs1ipy5hu4
- champagne problems (Album: 